# 04 - MODELO ALTERNATIVO: Random Forest con Pipeline (One-Hot)

## Entrega Final - Proyecto Modelos y Simulación de Sistemas

### Integrantes del equipo
| Nombre completo | Programa |
| :--- | :--- |
| Cristian David Diez Lopez | Ingeniería de Sistemas |
| Rafael Ángel Alemán Castillo | Ingeniería de Sistemas |
| Jonatan Romero Arrieta | Ingeniería de Sistemas |

---

### Descripción del enfoque (Aproximación Distinta 2)

Este notebook implementa una estrategia **completamente distinta** al basarse en la arquitectura clásica de Scikit-Learn.
* **Preprocesamiento (Pipeline Completo):** En lugar de usar el manejo nativo de categorías, aplicamos **One-Hot Encoding** explícito y escalado de variables numéricas (**StandardScaler**).
* **Modelo:** Se utiliza un **RandomForestClassifier**, un modelo de ensamble (Bagging) que contrasta con el Boosting (CatBoost) de la solución principal.

In [ ]:
!pip install --force-reinstall pandas==2.2.2


  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.4 MB/s eta 0:00:00
Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17

## 1. Definición del Pipeline de Preprocesamiento

A diferencia de los otros notebooks, aquí construimos un **Pipeline de Scikit-Learn** para asegurar un tratamiento riguroso de los datos y evitar el "data leakage":

1.  **Transformación Numérica (`num_transformer`):**
    * **Imputación:** Se usa la **mediana** (`SimpleImputer`) para rellenar huecos.
    * **Escalado:** Se aplica **StandardScaler** para normalizar los datos (media 0, desviación 1), lo cual es crucial cuando se combinan distancias o varianzas.

2.  **Transformación Categórica (`cat_transformer`):**
    * **Imputación:** Se rellenan los nulos con una constante `"missing"`.
    * **Codificación:** Se aplica **OneHotEncoder** con `handle_unknown='ignore'`. Esto explota las variables categóricas en múltiples columnas binarias (0/1), una representación necesaria para modelos como Random Forest que no manejan texto nativamente.

## 2. Definición y Entrenamiento del Modelo Random Forest

El modelo seleccionado es un **RandomForestClassifier** integrado directamente en el pipeline.
* **n_estimators=100:** Creamos un bosque de 100 árboles de decisión.
* **max_depth=10:** Limitamos la profundidad para controlar el sobreajuste, dado que el One-Hot Encoding incrementa significativamente la dimensionalidad de los datos de entrada.
* **n_jobs=-1:** Utilizamos todos los núcleos del procesador para paralelizar el entrenamiento.

In [ ]:
# ===============================================
# 04 - MODELO ONE-HOT + RANDOM FOREST
# ===============================================

import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


# ---------------------------
# Función de limpieza
# ---------------------------
def limpiar_para_onehot(df):
    df = df.copy()

    # Quitar ID
    if "ID" in df.columns:
        df.drop(columns=["ID"], inplace=True)

    # Mapear target
    if "RENDIMIENTO_GLOBAL" in df.columns:
        df["RENDIMIENTO_GLOBAL"] = (
            df["RENDIMIENTO_GLOBAL"]
            .astype(str)
            .str.strip()
            .str.lower()
        )

        mapa = {
            "bajo": 0,
            "medio-bajo": 1,
            "medio bajo": 1,
            "medio-alto": 2,
            "medio alto": 2,
            "alto": 3
        }

        df["RENDIMIENTO_GLOBAL"] = df["RENDIMIENTO_GLOBAL"].map(mapa)

        df["RENDIMIENTO_GLOBAL"] = df["RENDIMIENTO_GLOBAL"].fillna(
            df["RENDIMIENTO_GLOBAL"].mode()[0]
        )

    return df


# ---------------------------
# Cargar y limpiar datos
# ---------------------------
train = pd.read_csv("train.csv")
train = limpiar_para_onehot(train)

X = train.drop(columns=["RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]

cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(include=["int64", "float64"]).columns


# ---------------------------
# Preprocesador: Imputación + OneHot
# ---------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),

        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent"))
        ]), num_cols)
    ]
)


# ---------------------------
# Pipeline con modelo
# ---------------------------
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=300,
        max_depth=25,
        random_state=42,
        n_jobs=-1
    ))
])


# ---------------------------
# Split
# ---------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# ---------------------------
# Entrenar
# ---------------------------
pipeline.fit(X_train, y_train)

# Validar
pred = pipeline.predict(X_val)
print("Accuracy:", accuracy_score(y_val, pred))


# ---------------------------
# Predicción final
# ---------------------------
test = pd.read_csv("test.csv")
ids = test["ID"].copy()
test = limpiar_para_onehot(test)

test_pred = pipeline.predict(test)

reverse_map = {0: "bajo", 1: "medio-bajo", 2: "medio-alto", 3: "alto"}
test_pred = pd.Series(test_pred).map(reverse_map)

submission = pd.DataFrame({
    "ID": ids,
    "RENDIMIENTO_GLOBAL": test_pred
})

submission.to_csv("submission_04.csv", index=False)
print("Archivo submission_04.csv generado!")



Accuracy: 0.41031201313739524
Archivo submission_04.csv generado!
